In [18]:
import difflib
import heapq
import os
import json

In [28]:
txt_file_directory = r"D:\Code\DataSet\chatglm_llm_fintech_raw_dataset\alltxt"

In [14]:
def string_similar(s1, s2):
    return difflib.SequenceMatcher(None, s1, s2).quick_ratio()


In [34]:
def compute_similarity(clean_file_name, q, years):
    base_score = string_similar(clean_file_name, q)
    parts = clean_file_name.split("__")
    company_full_name, company_short_name, file_year = parts[0], parts[2], parts[3][:4]

    company_in_q = company_full_name in q or company_short_name in q
    year_in_q = file_year in years

    if not company_in_q:
        return 0.5 * base_score
    if not year_in_q:
        return 0.25 + 0.5 * base_score
    return 0.5 + 0.5 * base_score

In [39]:
def find_most_related_file(q, file_directory):
    # 统计问题中涉及的年份个数
    years = []
    for year in ["2019", "2020", "2021"]:  # 初赛问题中的年份只涉及这三个
        if year in q:
            years.append(year)

    similarities = []   # 用于存储每个文件与问题相似度
    # 遍历文件夹中的每一个文件
    for file_name in os.listdir(file_directory):
        if file_name.endswith(".txt"):
            clean_file_name = "__".join(file_name.split("__")[1:])
            file_path = os.path.join(file_directory, file_name)
            # 计算文件名与问题之间的相似度
            similarity = compute_similarity(clean_file_name, q, years)
            # 使用堆来存储最相关的 n 个文件，通过将相似度取负，实际上是创建了一个“大根堆”
            heapq.heappush(similarities, (-similarity, file_path))

    # 获取与问题最相关的 n 个文件
    n = len(years)
    most_related_files = [(abs(similarity), file_path) for similarity, file_path in [heapq.heappop(similarities) for _ in range(n)]]
    return most_related_files

In [40]:
find_most_related_file("2019年四方科技电子信箱是什么?", txt_file_directory)

[(0.65,
  'D:\\Code\\DataSet\\chatglm_llm_fintech_raw_dataset\\alltxt\\2020-04-16__四方科技集团股份有限公司__603339__四方科技__2019年__年度报告.txt')]

In [41]:
def get_questions_from_file(file_path):
    """从 JSON 文件中获取问题列表"""
    questions = []
    with open(file_path, 'r', encoding="utf-8") as fp:
        for line in fp.readlines():
            line = line.strip()
            if line:
                questions.append(json.loads(line)["question"])
    return questions

In [42]:
def main():
    questions = get_questions_from_file("初赛/test_questions.json")

    for q in questions:
        related_files = find_most_related_file(q, txt_file_directory)
        print(f"Question: {q}")
        print("Related files:")
        for file in related_files:
            print(f"- similarity: {file[0]}, file path: {file[1]}")
        print()

In [43]:
 main()


Question: 能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？
Related files:
- similarity: 0.717391304347826, file path: D:\Code\DataSet\chatglm_llm_fintech_raw_dataset\alltxt\2021-04-23__金宇生物技术股份有限公司__600201__生物股份__2020年__年度报告.txt

Question: 请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业情况进行分析。
Related files:
- similarity: 0.6428571428571428, file path: D:\Code\DataSet\chatglm_llm_fintech_raw_dataset\alltxt\2020-03-12__江阴江化微电子材料股份有限公司__603078__江化微__2019年__年度报告.txt

Question: 2019年四方科技电子信箱是什么?
Related files:
- similarity: 0.65, file path: D:\Code\DataSet\chatglm_llm_fintech_raw_dataset\alltxt\2020-04-16__四方科技集团股份有限公司__603339__四方科技__2019年__年度报告.txt

Question: 研发费用对公司的技术创新和竞争优势有何影响？
Related files:

Question: 康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？
Related files:
- similarity: 0.6772151898734178, file path: D:\Code\DataSet\chatglm_llm_fintech_raw_dataset\alltxt\2021-03-29__康希诺生物股份公司__688185__康希诺__2020年__年度报告.txt

Question: 苏州科达在2020年的博士及以上人员数量是多少?
Related files:
- similarity: 0.63432835820